## libraries and function 

In [8]:
# !pip install impyute
!pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import  SoftImpute
import pandas as pd
import time 
# from missingpy import MissForest

     |████████████████████████████████| 22.3 MB 50.7 MB/s 
     |████████████████████████████████| 2.7 MB 30.6 MB/s 
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=604621816f8833d542b010ca54161762cd4ba9cba8cdb93257e1f12a77d553a4
  Stored in directory: /root/.cache/pip/wheels/72/21/a8/a045cacd9838abd5643f6bfa852c0796a99d6b1494760494e0
Successfully built knnimpute
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


### MLE estimation function 

In [9]:
def diag_term(X,i):
  arr0 = X[:,i].flatten()
  arr = arr0[~np.isnan(arr0)]
  return np.var(arr)

def musMLE(X,y,G):
    n,p = X.shape[0], X.shape[1]
    f = lambda g: np.nanmean(X[y==g,:],axis=0)
    musMLE = np.array([f(g) for g in range(G)])    
    return musMLE.T

def Smle(X,y,musMLE,g):
    '''
    function to compute the covariance matrix for the g-th class
    X: input, should be a numpy array
    y: label
    G: number of classes
    g: class index
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G 
    '''
    epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
    Xg, yg = X[y==g,:], y[y==g]
    n,p = Xg.shape[0], Xg.shape[1] 
 
    S = np.diag([diag_term(Xg,i) for i in range(p)]) 

    for i in range(p):      
      for j in range(i):
        if ((S[i,i] == 0.) | (S[j,j] == 0.)):
          S[i,j] = S[j,i] = 0.
          continue

        mat = Xg[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat, y_arr = mat[idx], yg[idx]
        A = mg = len(y_arr) 

        s11 = mg*np.var(mat[:,0])
        s22 = mg*np.var(mat[:,1])
        s12 = sum((mat[:,0]-musMLE[i,g])*(mat[:,1]-musMLE[j,g]))
        B = S[i,i]*S[j,j]*A - s22 * S[i,i] - s11 * S[j,j]
        coefficient = [-A, s12, B, s12*S[i,i]*S[j,j]]
        r = np.roots(coefficient)
        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0
 
        if len(r)>1:
          condi_var = S[j,j] - r**2/S[i,i]
          eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1: 
          if sum(r==0.0) == len(r):
            r = 0.
          else:  
            w = np.cov(mat, rowvar=False)  
            #r = r[w[0,1]*r>=0]
            r = r[np.abs(r-w[0,1]).argmin()] # select r that is closet to w[0,1] 

        S[i,j] = S[j,i] = r
    return S

## Individual compute error function 

In [10]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
        np.linalg.norm(S_est.flatten().flatten()-S.flatten())/S.size]  
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape) 

In [11]:
def compute_err_mle(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  run_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])
    # MLEs approach
    start = time.time()
    mus_mle = musMLE(Xtr_nan,y,G)
    S_mle = np.array([Smle(Xtr_nan,y,mus_mle, g) for g in range(G)])   
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start  
    e_rate.append(mle_err)
    run_time.append(mle_time)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate), np.mean(np.asarray(run_time))

In [12]:
def compute_err_soft(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  run_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    print('mus soft impute', mus_softimpute)
    print('S soft impute', S_softimpute)
    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start
    e_rate.append(softimpute_err)
    run_time.append(softimpute_time)

  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate),np.mean(np.asarray(run_time))

### FASHION MNIST

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds

fashion_mnist = tf.keras.datasets.fashion_mnist
(Xtrain, ytrain), (Xtest, ytest) = fashion_mnist.load_data()
Xtrain = Xtrain.astype(float).reshape((60000,784))
Xtest = Xtest.astype(float).reshape((10000,784))

X = np.vstack((Xtrain, Xtest))
y = np.hstack((ytrain, ytest))

# set random seed and shuffle the data
np.random.seed(1)
idx = np.arange(len(y))
np.random.shuffle(idx)
X, y = X[idx,:], y[idx]  

X.shape, y.shape 

4431872/4422102 [==============================] - 0s 0us/step


((70000, 784), (70000,))

In [ ]:
G = 10
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
print('fashion mnist: soft - dper error')
print(mle_err.round(3))

In [ ]:
G = 10
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
print('fashion mnist: soft - impute error')
print(soft_err.round(3))

array([[2.89677844e-05, 6.16749044e+03],
       [3.95722202e-05, 5.05816724e+03],
       [5.04169202e-05, 4.26305263e+03],
       [5.89859441e-05, 3.49414068e+03],
       [8.19900695e-05, 2.91714304e+03]])